In [79]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [80]:
person = 'Kai'
data = pd.read_csv(f'{person}/autosleep.csv')
data['date'] = pd.to_datetime(data['ISO8601'], errors='coerce')
data['date'] = data['date'].apply(lambda x: x.tz_localize(None).normalize() if x is not pd.NaT else pd.NaT)
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
data['inBed_minutes'] = pd.to_timedelta(data['inBed']).dt.total_seconds() / 60
data['day_of_week'] = data['date'].dt.dayofweek
data['is_weekend'] = data['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
data['date'][len(data) - 10:]

1    2024-02-18
2    2024-02-19
3    2024-02-20
4    2024-02-21
5    2024-02-22
6    2024-02-23
7    2024-02-24
8    2024-02-25
9    2024-02-26
10   2024-02-27
Name: date, dtype: datetime64[ns]

In [81]:
def rename_columns(df):
  column_mappings = {
      'Name': 'name',
      'Date': 'date',
      'Grogginess': 'grogginess',
      'How many alarms did you set?': 'num_alarms',
      'What time did you set your first alarm for?': 'first_alarm',
      'Did you take sleep-aiding medicine (not weed, ie melatonin/antihystamine)?': 'sleep_medicine',
      "What was the temperature when you woke up? (Don't include °F, just the number)": 'waking_temp',
      'Were you intoxicated when you went to sleep?': 'intoxicated',
      'Were you sick when you went to sleep?': 'sick',
      'Did you eat within an hour of going to bed?': 'eat_before_bed',
      'Did you sleep alone?': 'sleep_alone',
      'Did you sleep in your own bed/room?': 'own_bed',
      'How stressed were you last night?': 'stress',
      'Did you use your phone before going to sleep?': 'phone_before_bed',
      "When was the latest you ingested caffeine before going to bed? (Don't answer if N/A)": 'caffeine_before_bed'
  }
  
  df_renamed = df.rename(columns=column_mappings)
  return df_renamed

In [82]:
suppl = rename_columns(pd.read_csv('form.csv'))

suppl['date'] = pd.to_datetime(suppl['date'], format='%m/%d/%Y').dt.normalize()


def time_to_minutes(time_str):
  if pd.isna(time_str) or time_str == '':
      return 0
  time = pd.to_datetime(time_str, format='%I:%M:%S %p')
  return time.hour * 60 + time.minute

suppl['first_alarm'] = suppl['first_alarm'].apply(time_to_minutes)
suppl['caffeine_before_bed'] = suppl['caffeine_before_bed'].apply(time_to_minutes)
lag_periods = 1
suppl['grogginess_lag1'] = suppl['grogginess'].shift(lag_periods)
suppl = suppl[suppl['name'] == person].fillna(0)
suppl

,Timestamp,name,date,grogginess,num_alarms,first_alarm,sleep_medicine,waking_temp,intoxicated,sick,eat_before_bed,sleep_alone,own_bed,stress,phone_before_bed,caffeine_before_bed,grogginess_lag1
0,2/19/2024 15:34:32,Kai,2024-02-18,5,1,675,0,74.0,0,0,0,1,1,3,1,0,0.0
1,2/19/2024 15:35:20,Kai,2024-02-19,6,1,510,1,74.0,0,0,0,1,1,3,1,0,5.0
4,2/20/2024 8:48:05,Kai,2024-02-20,3,1,525,0,72.0,0,0,0,1,1,4,1,0,3.0
6,2/21/2024 8:40:51,Kai,2024-02-21,6,4,390,0,74.0,0,0,0,1,1,5,1,0,6.0
8,2/22/2024 9:15:05,Kai,2024-02-22,6,6,361,1,74.0,0,0,0,1,1,6,1,0,5.0
11,2/23/2024 8:59:04,Kai,2024-02-23,7,7,390,1,73.0,0,0,0,1,1,5,1,0,8.0
14,2/24/2024 9:16:16,Kai,2024-02-24,6,2,540,0,74.0,0,0,0,1,1,6,1,600,4.0
16,2/25/2024 12:49:50,Kai,2024-02-25,3,0,0,1,74.0,0,0,0,1,1,5,1,1380,8.0


In [83]:
merged_data = pd.merge(suppl, data, on='date', how='inner').fillna(0)

merged_data.head(10)


,Timestamp,name,date,grogginess,num_alarms,first_alarm,sleep_medicine,waking_temp,intoxicated,sick,...,respMin,respMax,tags,notes,year,month,day,inBed_minutes,day_of_week,is_weekend
0,2/19/2024 15:34:32,Kai,2024-02-18,5,1,675,0,74.0,0,0,...,0.0,0.0,0.0,0.0,2024,2,18,495.0,6,1
1,2/19/2024 15:35:20,Kai,2024-02-19,6,1,510,1,74.0,0,0,...,0.0,0.0,0.0,0.0,2024,2,19,594.0,0,0
2,2/20/2024 8:48:05,Kai,2024-02-20,3,1,525,0,72.0,0,0,...,0.0,0.0,0.0,0.0,2024,2,20,540.0,1,0
3,2/21/2024 8:40:51,Kai,2024-02-21,6,4,390,0,74.0,0,0,...,0.0,0.0,0.0,0.0,2024,2,21,539.0,2,0
4,2/22/2024 9:15:05,Kai,2024-02-22,6,6,361,1,74.0,0,0,...,0.0,0.0,0.0,0.0,2024,2,22,576.0,3,0
5,2/23/2024 8:59:04,Kai,2024-02-23,7,7,390,1,73.0,0,0,...,0.0,0.0,0.0,0.0,2024,2,23,448.0,4,0
6,2/24/2024 9:16:16,Kai,2024-02-24,6,2,540,0,74.0,0,0,...,0.0,0.0,0.0,0.0,2024,2,24,384.0,5,1
7,2/25/2024 12:49:50,Kai,2024-02-25,3,0,0,1,74.0,0,0,...,0.0,0.0,0.0,0.0,2024,2,25,614.0,6,1


In [84]:
features = ['sleepBPM', 'sleepBPMAvg7', 'dayBPM', 'dayBPMAvg7', 'wakingBPM', 'wakingBPMAvg7', 
            'hrv', 'hrvAvg7', 'sleepHRV', 'sleepHRVAvg7', 'SpO2Avg', 'SpO2Min', 'SpO2Max', 
            'respAvg', 'respMin', 'respMax', 'year', 'month', 'day', 
            'inBed_minutes', 'day_of_week', 'is_weekend', ]
features += ['first_alarm', 'num_alarms', 'sleep_medicine', 'waking_temp', 'intoxicated', 'sick', 'eat_before_bed', 'sleep_alone', 'own_bed', 'stress', 'phone_before_bed', 'caffeine_before_bed', 'grogginess_lag1']

target = ['grogginess']

# merged_data[features].head().to_csv('out.csv')
# from sklearn.model_selection import train_test_split

X = merged_data[features]  # Features
y = merged_data[target]  # Assuming you have a target variable

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# split_point = int(len(X) * 0.9)
split_point = len(X) - 1
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]

X_test.head()

,sleepBPM,sleepBPMAvg7,dayBPM,dayBPMAvg7,wakingBPM,wakingBPMAvg7,hrv,hrvAvg7,sleepHRV,sleepHRVAvg7,...,waking_temp,intoxicated,sick,eat_before_bed,sleep_alone,own_bed,stress,phone_before_bed,caffeine_before_bed,grogginess_lag1
7,68.0,65.7,81.6,84.0,66.0,61.1,105,103,56,56,...,74.0,0,0,0,1,1,5,1,1380,8.0


In [85]:
from torch.utils.data import TensorDataset, DataLoader

np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(42)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)
batch_size = 32  # Adjust based on your computational resources

training_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(dataset=training_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)


In [86]:
class GrogginessModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GrogginessModel, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size) 
        self.layer2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = self.layer2(x)
        return x

input_size = len(features) 
hidden_size = 100  
output_size = 1
model = GrogginessModel(input_size, hidden_size, output_size)
len(features)

35

In [87]:
criterion = nn.MSELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001) 
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [88]:
def evaluate_model(model, test_loader, criterion):
  model.eval()  # Set the model to evaluation mode
  total_loss = 0.0
  with torch.no_grad():  # No need to track gradients
    for inputs, targets in test_loader:
      outputs = model(inputs)
      loss = criterion(outputs, targets)
      total_loss += loss.item() * inputs.size(0)
  avg_loss = total_loss / len(test_loader.dataset)
  return avg_loss
  
class EarlyStopping:
  def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
    self.patience = patience
    self.verbose = verbose
    self.counter = 0
    self.best_score = None
    self.early_stop = False
    self.val_loss_min = np.Inf
    self.delta = delta
    self.path = path

  def __call__(self, val_loss, model):
    score = -val_loss

    if self.best_score is None:
      self.best_score = score
      self.save_checkpoint(val_loss, model)
    elif score < self.best_score + self.delta:
      self.counter += 1
      print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
      if self.counter >= self.patience:
          self.early_stop = True
    else:
      self.best_score = score
      self.save_checkpoint(val_loss, model)
      self.counter = 0

  def save_checkpoint(self, val_loss, model):
    if self.verbose:
      print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
    torch.save(model.state_dict(), self.path)
    self.val_loss_min = val_loss


In [89]:
epochs = 10000
early_stopping = EarlyStopping(patience=1000, verbose=True)

for epoch in range(epochs):
    model.train()  # Set the model to training mode
    val_loss = evaluate_model(model, test_loader, criterion)
    early_stopping(val_loss, model)
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, targets)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize
        
        running_loss += loss.item() * inputs.size(0)  # Total loss for the batch

    epoch_loss = running_loss / len(train_loader.dataset)  # Average loss for the epoch

    if epoch % 100 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}')


Validation loss decreased (inf --> 32.438931).  Saving model ...
Epoch 1/10000, Loss: 12477.6406
EarlyStopping counter: 1 out of 1000
EarlyStopping counter: 2 out of 1000
EarlyStopping counter: 3 out of 1000
EarlyStopping counter: 4 out of 1000
EarlyStopping counter: 5 out of 1000
EarlyStopping counter: 6 out of 1000
EarlyStopping counter: 7 out of 1000
EarlyStopping counter: 8 out of 1000
EarlyStopping counter: 9 out of 1000
EarlyStopping counter: 10 out of 1000
EarlyStopping counter: 11 out of 1000
EarlyStopping counter: 12 out of 1000
EarlyStopping counter: 13 out of 1000
EarlyStopping counter: 14 out of 1000
EarlyStopping counter: 15 out of 1000
EarlyStopping counter: 16 out of 1000
EarlyStopping counter: 17 out of 1000
EarlyStopping counter: 18 out of 1000
EarlyStopping counter: 19 out of 1000
EarlyStopping counter: 20 out of 1000
EarlyStopping counter: 21 out of 1000
EarlyStopping counter: 22 out of 1000
EarlyStopping counter: 23 out of 1000
EarlyStopping counter: 24 out of 1000


In [90]:
print(features)

['sleepBPM', 'sleepBPMAvg7', 'dayBPM', 'dayBPMAvg7', 'wakingBPM', 'wakingBPMAvg7', 'hrv', 'hrvAvg7', 'sleepHRV', 'sleepHRVAvg7', 'SpO2Avg', 'SpO2Min', 'SpO2Max', 'respAvg', 'respMin', 'respMax', 'year', 'month', 'day', 'inBed_minutes', 'day_of_week', 'is_weekend', 'first_alarm', 'num_alarms', 'sleep_medicine', 'waking_temp', 'intoxicated', 'sick', 'eat_before_bed', 'sleep_alone', 'own_bed', 'stress', 'phone_before_bed', 'caffeine_before_bed', 'grogginess_lag1']


In [91]:
realtime_data_values = X_test[features].iloc[0]
# realtime_data_values['wakingBPM'] = 100
# print(realtime_data_values)
realtime_data_values = realtime_data_values.tolist()


from datetime import date
year, month, day = [int(x) for x in X_test[['year', 'month', 'day']].iloc[0].tolist()]

date_of_test = str(date(year, month, day))
# print(date_of_test)
original_grog = float(suppl[suppl['date'] == date_of_test]['grogginess'])
print(original_grog)

3.0


/var/folders/m_/j2q_c7f556j5syqrs041q30m0000gn/T/ipykernel_54575/2047264419.py:12: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  original_grog = float(suppl[suppl['date'] == date_of_test]['grogginess'])


In [92]:

realtime_data_tensor = torch.tensor([realtime_data_values], dtype=torch.float32)  # Wrap in a list to keep dimensions

model.eval()  # Set the model to evaluation mode
with torch.no_grad():
  predicted_grogginess = model(realtime_data_tensor)
print(predicted_grogginess[0])
print(original_grog)


tensor([-2.0867])
3.0


In [93]:

avg_test_loss = evaluate_model(model, test_loader, criterion)
print(f'Average test loss: {avg_test_loss:.4f}')

Average test loss: 25.8741
